In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# E2E ML on GCP: MLOps stage 3 : formalization: get started with BigQuery ML pipeline components

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage3/get_started_with_bqml_pipeline_components.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/ai/platform/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage3/get_started_with_bqml_pipeline_components.ipynb">
      Open in Google Cloud Notebooks
    </a>
  </td>
</table>
<br/><br/><br/>

## Overview


This tutorial demonstrates how to use Vertex AI for E2E MLOps on Google Cloud in production. This tutorial covers stage 3 : formalization: get started with BigQuery ML pipeline components.

### Dataset

The dataset used for this tutorial is the Penguins dataset from [BigQuery public datasets](https://cloud.google.com/bigquery/public-data). The version of the dataset predicts the species.

### Objective

In this tutorial, you learn how to use prebuilt `Google Cloud Pipeline Components` for `BigQuery ML`.

This tutorial uses the following Google Cloud ML services:

- `BigQuery ML`
- `Google Cloud Pipeline Components`
- `Vertex AI Dataset, Model and Endpoint` resources
- `Vertex AI Prediction`

The steps performed include:

- Construct a pipeline for:
    - Training BigQuery ML model.
    - Evaluating the BigQuery ML model.
    - Exporting the BigQuery ML model.
    - Importing the BigQuery ML model to a Vertex AI model.
    - Deploy the Vertex AI model.
- Execute a Vertex AI pipeline.
- Make a prediction with the deployed Vertex AI model.

## Installations

Install *one time* the packages for executing the MLOps notebooks.

In [ ]:
ONCE_ONLY = False
if ONCE_ONLY:
    ! pip3 install -U tensorflow==2.5 $USER_FLAG
    ! pip3 install -U tensorflow-data-validation==1.2 $USER_FLAG
    ! pip3 install -U tensorflow-transform==1.2 $USER_FLAG
    ! pip3 install -U tensorflow-io==0.18 $USER_FLAG
    ! pip3 install --upgrade google-cloud-aiplatform[tensorboard] $USER_FLAG
    ! pip3 install --upgrade google-cloud-pipeline-components $USER_FLAG
    ! pip3 install --upgrade google-cloud-bigquery $USER_FLAG
    ! pip3 install --upgrade google-cloud-logging $USER_FLAG
    ! pip3 install --upgrade apache-beam[gcp] $USER_FLAG
    ! pip3 install --upgrade pyarrow $USER_FLAG
    ! pip3 install --upgrade cloudml-hypertune $USER_FLAG
    ! pip3 install --upgrade kfp $USER_FLAG

### Restart the kernel

Once you've installed the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append the timestamp onto the name of resources you create in this tutorial.

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you initialize the Vertex SDK for Python, you specify a Cloud Storage staging bucket. The staging bucket is where all the data associated with your dataset and model resources are retained across sessions.

Set the name of your Cloud Storage bucket below. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al $BUCKET_NAME

#### Service Account

**If you don't know your service account**, try to get your service account using `gcloud` command by executing the second cell below.

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your GCP project id from gcloud
    shell_output = !gcloud auth list 2>/dev/null
    SERVICE_ACCOUNT = shell_output[2].strip()
    print("Service Account:", SERVICE_ACCOUNT)

#### Set service account access for Vertex AI Pipelines

Run the following commands to grant your service account access to read and write pipeline artifacts in the bucket that you created in the previous step -- you only need to run these once per service account.

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_NAME

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_NAME

### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries and define constants

In [ ]:
import google.cloud.aiplatform as aip

In [ ]:
import json

from kfp import dsl
from kfp.v2 import compiler
from kfp.v2.dsl import component

#### Import BigQuery

Import the BigQuery package into your Python environment.

In [ ]:
from google.cloud import bigquery

#### Import TensorFlow

Import the TensorFlow package into your Python environment.

In [ ]:
import tensorflow as tf

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [ ]:
aip.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_NAME)

### Create BigQuery client

Create the BigQuery client.

In [ ]:
bqclient = bigquery.Client()

#### Set hardware accelerators

You can set hardware accelerators for prediction.

Set the variable `DEPLOY_GPU/DEPLOY_NGPU` to use a container image supporting a GPU and the number of GPUs allocated to the virtual machine (VM) instance. For example, to use a GPU container image with 4 Nvidia Telsa K80 GPUs allocated to each VM, you would specify:

    (aip.AcceleratorType.NVIDIA_TESLA_K80, 4)

Otherwise specify `(None, None)` to use a container image to run on a CPU.

Learn more [here](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators) hardware accelerator support for your region

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (aip.gapic.AcceleratorType.NVIDIA_TESLA_K80, 1)

#### Set pre-built containers

Set the pre-built Docker container image for prediction.

- Set the variable `TF` to the TensorFlow version of the container image. For example, `2-1` would be version 2.1, and `1-15` would be version 1.15. The following list shows some of the pre-built images available:


For the latest list, see [Pre-built containers for prediction](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers).

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2.5".replace(".", "-")

if TF[0] == "2":
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    REGION.split("-")[0], DEPLOY_VERSION
)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU)

#### Location of BigQuery training data.

Now set the variable `IMPORT_FILE` to the location of the data table in BigQuery.

In [ ]:
IMPORT_FILE = "bq://bigquery-public-data.ml_datasets.penguins"
BQ_TABLE = "bigquery-public-data.ml_datasets.penguins"

### Construct pipeline for BigQuery ML training, exporting and deployment

Next, construct the pipeline with the following tasks:

- Create a BigQuery schema for the dataset.
- Train a BigQuery ML model.
- Evaluate the BigQuery ML model.
- Make a batch prediction with the BigQuery ML model. *Note:* The same data (table) is used for both training and prediction.
- Export the BigQuery ML model.
- Import the exported model artifacts to a Vertex AI Model resource.
- Deploy the Vertex AI Model resource to a Vertex AI Endpoint resource.

In [ ]:
PIPELINE_ROOT = f"{BUCKET_NAME}/bq_query"


@dsl.pipeline(name="bq-hello-world", pipeline_root=PIPELINE_ROOT)
def pipeline(
    bq_table: str,
    label: str,
    dataset: str,
    model: str,
    artifact_uri: str,
    min_trials: int,
    deploy_image: str,
    machine_type: str,
    min_replica_count: int,
    max_replica_count: int,
    display_name: str,
    accelerator_type: str = "",
    accelerator_count: int = 0,
    project: str = PROJECT_ID,
    location: str = "US",
    region: str = "us-central1",
):
    import google_cloud_pipeline_components.experimental.bigquery as gcc_bq
    from google_cloud_pipeline_components import aiplatform as gcc_aip

    bq_dataset = gcc_bq.BigqueryQueryJobOp(
        project=project, location="US", query=f"CREATE SCHEMA {dataset}"
    )

    bq_model = gcc_bq.BigqueryCreateModelJobOp(
        project=project,
        location=location,
        query=f"CREATE OR REPLACE MODEL {dataset}.{model} OPTIONS (model_type='dnn_classifier', labels=['{label}'], num_trials={min_trials}) AS SELECT * FROM `{bq_table}` WHERE body_mass_g IS NOT NULL AND sex IS NOT NULL",
    ).after(bq_dataset)

    # bq_eval = gcc_bq.BigqueryEvaluateModelJobOp(
    # project=PROJECT_ID,
    # location="US",
    # model_name="bqml_tutorial.penguins_model",
    # ).after(bq_model)

    bq_eval = gcc_bq.BigqueryQueryJobOp(
        project=project,
        location=location,
        query=f"SELECT * FROM ML.EVALUATE(MODEL {dataset}.{model}) ORDER BY  roc_auc desc LIMIT 1",
    ).after(bq_model)

    bq_predict = gcc_bq.BigqueryPredictModelJobOp(
        project=project,
        location=location,
        model_name=f"{dataset}.{model}",
        table_name=f"`{bq_table}`",
        # query_statement=f"SELECT * EXCEPT ({label}) FROM {bq_table} WHERE body_mass_g IS NOT NULL AND sex IS NOT NULL"
        job_configuration_query={
            "destinationTable": {
                "projectId": PROJECT_ID,
                "datasetId": "bqml_tutorial",
                "tableId": "results_1",
            }
        },
    ).after(bq_model)

    bq_export = gcc_bq.BigqueryExportModelJobOp(
        project=project,
        location=location,
        model_name=f"{project}.{dataset}.{model}",
        model_destination_path=artifact_uri,
    ).after(bq_model)

    model_upload = gcc_aip.ModelUploadOp(
        display_name=display_name,
        artifact_uri=artifact_uri,
        serving_container_image_uri=deploy_image,
        project=project,
        location=region,
    ).after(bq_export)

    endpoint = gcc_aip.EndpointCreateOp(
        project=project,
        location=region,
        display_name=display_name,
    ).after(model_upload)

    deploy_model = gcc_aip.ModelDeployOp(
        model=model_upload.outputs["model"],
        endpoint=endpoint.outputs["endpoint"],
        dedicated_resources_min_replica_count=min_replica_count,
        dedicated_resources_max_replica_count=max_replica_count,
        dedicated_resources_machine_type=machine_type,
        dedicated_resources_accelerator_type=accelerator_type,
        dedicated_resources_accelerator_count=accelerator_count,
        traffic_split={"0": 100},
    )

### Compile and execute the BigQuery ML train, export and deploy pipeline

Next, you compile the pipeline and then execute it. The pipeline takes the following parameters, which are passed as the dictionary `parameter_values`:

- `bq_table`: The BigQuery dataset to train on.
- `label`: The corresponding label for the BigQuery dataset.
- `dataset`: The BigQuery dataset component name.
- `model`: The BigQuery model component name.
- `artifact_uri`: The Cloud Storage location to export the BigQuery model artifacts.
- `min_trials`: If greater than one, will perform hyperparameter tuning for the specified number of trials using the Vertex AI Vizier service.
- `deploy_image`: The container image for serving predictions.
- `machine_type`: The VM for serving predictions.
- `min_replica_count`/`max_replica_count`: The number of virtual machines for auto-scaling predictions.
- `accelerator_type`: The type of HW accelerators -- if any.
- `accelerator_count`: The number of HW accelerators -- if any.
- `display_name`: Display name for Vertex AI Model and Endpoint resources.
- `project`: The project ID
- `location`: The location

In [ ]:
MODEL_DIR = BUCKET_NAME + "/bqmodel"

compiler.Compiler().compile(pipeline_func=pipeline, package_path="bqml.json")

pipeline = aip.PipelineJob(
    display_name="bqml",
    template_path="bqml.json",
    pipeline_root=PIPELINE_ROOT,
    parameter_values={
        "bq_table": BQ_TABLE,
        "label": "species",
        "dataset": "bqml_tutorial",
        "model": "penguins_model",
        "artifact_uri": MODEL_DIR,
        "min_trials": 2,
        "deploy_image": DEPLOY_IMAGE,
        "display_name": "penguins",
        "machine_type": "n1-standard-4",
        "min_replica_count": 1,
        "max_replica_count": 1,
        "accelerator_type": DEPLOY_GPU.name,
        "accelerator_count": DEPLOY_NGPU,
        "project": PROJECT_ID,
        "location": "US",
    },
    enable_caching=False,
)

pipeline.run()

! rm -rf bqml.json

### View the BigQuery ML pipeline results

In [ ]:
PROJECT_NUMBER = pipeline.gca_resource.name.split("/")[1]
print(PROJECT_NUMBER)


def print_pipeline_output(job, output_task_name):
    JOB_ID = job.name
    print(JOB_ID)
    for _ in range(len(job.gca_resource.job_detail.task_details)):
        TASK_ID = job.gca_resource.job_detail.task_details[_].task_id
        EXECUTE_OUTPUT = (
            PIPELINE_ROOT
            + "/"
            + PROJECT_NUMBER
            + "/"
            + JOB_ID
            + "/"
            + output_task_name
            + "_"
            + str(TASK_ID)
            + "/executor_output.json"
        )
        GCP_RESOURCES = (
            PIPELINE_ROOT
            + "/"
            + PROJECT_NUMBER
            + "/"
            + JOB_ID
            + "/"
            + output_task_name
            + "_"
            + str(TASK_ID)
            + "/gcp_resources"
        )
        if tf.io.gfile.exists(EXECUTE_OUTPUT):
            ! gsutil cat $EXECUTE_OUTPUT
            break
        elif tf.io.gfile.exists(GCP_RESOURCES):
            ! gsutil cat $GCP_RESOURCES
            break

    return EXECUTE_OUTPUT


print("bigquery-query-job")
artifacts = print_pipeline_output(pipeline, "bigquery-query-job")
print("\n\n")
print("bigquery-create-model-job")
artifacts = print_pipeline_output(pipeline, "bigquery-create-model-job")
print("\n\n")
print("bigquery-query-job-2")
artifacts = print_pipeline_output(pipeline, "bigquery-query-job-2")
print("\n\n")
print("bigquery-predict-model-job")
artifacts = print_pipeline_output(pipeline, "bigquery-predict-model-job")
print("\n\n")
print("bigquery-export-model-job")
artifacts = print_pipeline_output(pipeline, "bigquery-export-model-job")
print("\n\n")
print("model-upload")
artifacts = print_pipeline_output(pipeline, "model-upload")
print("\n\n")
print("endpoint-create")
artifacts = print_pipeline_output(pipeline, "endpoint-create")
print("\n\n")
output = !gsutil cat $artifacts
output = json.loads(output[0])
endpoint_id = output["artifacts"]["endpoint"]["artifacts"][0]["metadata"][
    "resourceName"
]
print("\n")
print(endpoint_id)
print("\n")
print("model-deploy")
artifacts = print_pipeline_output(pipeline, "model-deploy")
print("\n\n")

### Delete a pipeline job

After a pipeline job is completed, you can delete the pipeline job with the method `delete()`.  Prior to completion, a pipeline job can be canceled with the method `cancel()`.

In [ ]:
pipeline.delete()

## Load the saved model

Your model is stored in a TensorFlow SavedModel format in a Cloud Storage bucket. Now load it from the Cloud Storage bucket, and then you can do some things, like evaluate the model, and do a prediction.

To load, you use the TF.Keras `model.load_model()` method passing it the Cloud Storage path where the model is saved -- specified by `MODEL_DIR`.

In [ ]:
model = tf.keras.models.load_model(MODEL_DIR)

model.summary()

#### Load an endpoint

The 'Endpoint' initializer will load an endpoint from an endpoint identifier.

In [ ]:
endpoint = aip.Endpoint(endpoint_id)

#### Make prediction instances

Next, you prepare a prediction request using a synthetic example. The format for making a prediction request to an export BigQuery ML model is the same as the format for an AutoML model trained on the same data:

    { 'instances': [ {instance_1}, {instance_2}, ... ])

    instance -> { 'feature_1': value_1, 'feature_2': value_2, ... }

In [ ]:
INSTANCES = {
    "instances": [
        {
            "island": "DREAM",
            "culmen_length_mm": 36.6,
            "culmen_depth_mm": 18.4,
            "flipper_length_mm": 184.0,
            "body_mass_g": 3475.0,
            "sex": "FEMALE",
        }
    ]
}

### Make a prediction

Finally, you make the prediction request using Vertex AI Prediction service.

In [ ]:
response = endpoint.predict(INSTANCES)
print(response)

#### Delete the BigQuery model and dataset

Next, delete the BigQuery model and dataset.

In [ ]:
try:
    job = bqclient.delete_model("bqml_tutorial.penguins_model")
except:
    pass
job = bqclient.delete_dataset("bqml_tutorial", delete_contents=True)

# Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

- Dataset
- Pipeline
- Model
- Endpoint
- AutoML Training Job
- Batch Job
- Custom Job
- Hyperparameter Tuning Job
- Cloud Storage Bucket

In [ ]:
delete_all = True

if delete_all:
    # Delete the dataset using the Vertex dataset object
    try:
        if "dataset" in globals():
            dataset.delete()
    except Exception as e:
        print(e)

    # Delete the model using the Vertex model object
    try:
        if "model" in globals():
            model.delete()
    except Exception as e:
        print(e)

    # Delete the endpoint using the Vertex endpoint object
    try:
        if "endpoint" in globals():
            endpoint.undeploy_all()
            endpoint.delete()
    except Exception as e:
        print(e)

    # Delete the AutoML or Pipeline training job
    try:
        if "dag" in globals():
            dag.delete()
    except Exception as e:
        print(e)

    # Delete the custom training job
    try:
        if "job" in globals():
            job.delete()
    except Exception as e:
        print(e)

    # Delete the batch prediction job using the Vertex batch prediction object
    try:
        if "batch_predict_job" in globals():
            batch_predict_job.delete()
    except Exception as e:
        print(e)

    # Delete the hyperparameter tuning job using the Vertex hyperparameter tuning object
    try:
        if "hpt_job" in globals():
            hpt_job.delete()
    except Exception as e:
        print(e)

    if "BUCKET_NAME" in globals():
        ! gsutil rm -r $BUCKET_NAME